In [2]:
import torch     
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [3]:
torch.manual_seed(1) 
if torch.cuda.is_available():
    torch.cuda.manual_seed(torch.cuda.is_available()) 

In [4]:
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
train_loader = torch.utils.data.DataLoader(       
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(        
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
 
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
def train(epoch):      
    model.train()      
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():  
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)  
        optimizer.zero_grad()  
        output = model(data)   
        loss = F.nll_loss(output, target)                
        loss.backward()        
        optimizer.step()       
        if batch_idx % 10 == 0:        
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
 
def test():
    model.eval()      
    test_loss = 0     
    correct = 0       
    for data, target in test_loader:
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1] 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()  
 
    test_loss /= len(test_loader.dataset) 
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
if __name__ == "__main__":
     
    model = Net()  
    print(model)
    if torch.cuda.is_available():
        model.cuda()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  
        
    for epoch in range(1, 10 + 1):
        train(epoch)
        test()

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.347563
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.301364
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.257706
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.151477
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.818059
Train Epoch: 1 [5000/60000 (8%)]	Loss: 1.592572
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.325295
Train Epoch: 1 [7000/60000 (12%)]	Loss: 1.382306
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.172907
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.752725
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.788748
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.089534
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.821150
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.852748
Train Epoch: 1 [14000/

/home/arg/machine_learning/machine_learning/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/arg/machine_learning/machine_learning/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1066, Accuracy: 9686/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.313502
Train Epoch: 2 [1000/60000 (2%)]	Loss: 0.293890
Train Epoch: 2 [2000/60000 (3%)]	Loss: 0.388853
Train Epoch: 2 [3000/60000 (5%)]	Loss: 0.422823
Train Epoch: 2 [4000/60000 (7%)]	Loss: 0.261272
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.334303
Train Epoch: 2 [6000/60000 (10%)]	Loss: 0.263416
Train Epoch: 2 [7000/60000 (12%)]	Loss: 0.276361
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.198882
Train Epoch: 2 [9000/60000 (15%)]	Loss: 0.229756
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.198728
Train Epoch: 2 [11000/60000 (18%)]	Loss: 0.203378
Train Epoch: 2 [12000/60000 (20%)]	Loss: 0.390659
Train Epoch: 2 [13000/60000 (22%)]	Loss: 0.260245
Train Epoch: 2 [14000/60000 (23%)]	Loss: 0.332061
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.275290
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.204376
Train Epoch: 2 [17000/60000 (28%)]	Loss: 0.273890
Train Epoch: 2 [18000/60000 (30%)]	Loss: 0.230931
Train Ep

Train Epoch: 4 [42000/60000 (70%)]	Loss: 0.155124
Train Epoch: 4 [43000/60000 (72%)]	Loss: 0.125357
Train Epoch: 4 [44000/60000 (73%)]	Loss: 0.224123
Train Epoch: 4 [45000/60000 (75%)]	Loss: 0.197672
Train Epoch: 4 [46000/60000 (77%)]	Loss: 0.232093
Train Epoch: 4 [47000/60000 (78%)]	Loss: 0.263428
Train Epoch: 4 [48000/60000 (80%)]	Loss: 0.261717
Train Epoch: 4 [49000/60000 (82%)]	Loss: 0.115912
Train Epoch: 4 [50000/60000 (83%)]	Loss: 0.265197
Train Epoch: 4 [51000/60000 (85%)]	Loss: 0.198287
Train Epoch: 4 [52000/60000 (87%)]	Loss: 0.100310
Train Epoch: 4 [53000/60000 (88%)]	Loss: 0.161961
Train Epoch: 4 [54000/60000 (90%)]	Loss: 0.130236
Train Epoch: 4 [55000/60000 (92%)]	Loss: 0.133743
Train Epoch: 4 [56000/60000 (93%)]	Loss: 0.182237
Train Epoch: 4 [57000/60000 (95%)]	Loss: 0.094986
Train Epoch: 4 [58000/60000 (97%)]	Loss: 0.322633
Train Epoch: 4 [59000/60000 (98%)]	Loss: 0.151976

Test set: Average loss: 0.0562, Accuracy: 9834/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.1

Train Epoch: 7 [24000/60000 (40%)]	Loss: 0.228447
Train Epoch: 7 [25000/60000 (42%)]	Loss: 0.157217
Train Epoch: 7 [26000/60000 (43%)]	Loss: 0.181795
Train Epoch: 7 [27000/60000 (45%)]	Loss: 0.069558
Train Epoch: 7 [28000/60000 (47%)]	Loss: 0.204792
Train Epoch: 7 [29000/60000 (48%)]	Loss: 0.107505
Train Epoch: 7 [30000/60000 (50%)]	Loss: 0.185223
Train Epoch: 7 [31000/60000 (52%)]	Loss: 0.085287
Train Epoch: 7 [32000/60000 (53%)]	Loss: 0.116253
Train Epoch: 7 [33000/60000 (55%)]	Loss: 0.230900
Train Epoch: 7 [34000/60000 (57%)]	Loss: 0.207545
Train Epoch: 7 [35000/60000 (58%)]	Loss: 0.192607
Train Epoch: 7 [36000/60000 (60%)]	Loss: 0.110093
Train Epoch: 7 [37000/60000 (62%)]	Loss: 0.158772
Train Epoch: 7 [38000/60000 (63%)]	Loss: 0.100481
Train Epoch: 7 [39000/60000 (65%)]	Loss: 0.139742
Train Epoch: 7 [40000/60000 (67%)]	Loss: 0.142779
Train Epoch: 7 [41000/60000 (68%)]	Loss: 0.067337
Train Epoch: 7 [42000/60000 (70%)]	Loss: 0.155636
Train Epoch: 7 [43000/60000 (72%)]	Loss: 0.123739


Train Epoch: 10 [6000/60000 (10%)]	Loss: 0.109467
Train Epoch: 10 [7000/60000 (12%)]	Loss: 0.103138
Train Epoch: 10 [8000/60000 (13%)]	Loss: 0.050566
Train Epoch: 10 [9000/60000 (15%)]	Loss: 0.094595
Train Epoch: 10 [10000/60000 (17%)]	Loss: 0.112541
Train Epoch: 10 [11000/60000 (18%)]	Loss: 0.097658
Train Epoch: 10 [12000/60000 (20%)]	Loss: 0.138493
Train Epoch: 10 [13000/60000 (22%)]	Loss: 0.155266
Train Epoch: 10 [14000/60000 (23%)]	Loss: 0.192147
Train Epoch: 10 [15000/60000 (25%)]	Loss: 0.296806
Train Epoch: 10 [16000/60000 (27%)]	Loss: 0.103465
Train Epoch: 10 [17000/60000 (28%)]	Loss: 0.078712
Train Epoch: 10 [18000/60000 (30%)]	Loss: 0.131778
Train Epoch: 10 [19000/60000 (32%)]	Loss: 0.197350
Train Epoch: 10 [20000/60000 (33%)]	Loss: 0.286531
Train Epoch: 10 [21000/60000 (35%)]	Loss: 0.123550
Train Epoch: 10 [22000/60000 (37%)]	Loss: 0.224608
Train Epoch: 10 [23000/60000 (38%)]	Loss: 0.093264
Train Epoch: 10 [24000/60000 (40%)]	Loss: 0.077390
Train Epoch: 10 [25000/60000 (42%)]